In [19]:
import pandas as pd
import numpy as np
import random

In [20]:
CONFIG = {
	'sellFee' : 0.002,
    'buyFee' : 0.002,
    'taxRate' : 0.22,
	'phaseList' : [
        {
			'phaseNum' : 0,
            'phaseQqqPosition' : 1,
            'phaseTltPosition' : 0
		},
		{
			'phaseNum' : 1,
			'phaseCnt' : 1,
            'phasePeriodNum' : 1,
            'phasePeriodType' : 'm',
            'phaseThresholdRate' : 2,
            'phaseQqqPosition' : 0.1,
            'phaseTltPosition' : 0.9
		},
		{
			'phaseNum' : 2,
			'phaseCnt' : 1,
            'phasePeriodNum' : 1,
            'phasePeriodType' : 'm',
            'phaseThresholdRate' : 3,
            'phaseQqqPosition' : 0,
            'phaseTltPosition' : 1
		},
        {
            'phaseNum' : 3,
            'phaseCnt' : 4,
            'phasePeriodNum' : 2,
            'phasePeriodType' : 'm',
            'phaseThresholdRate' : 3,
            'phaseQqqPosition' : 0,
            'phaseTltPosition' : 1
        }
	]
}

In [21]:
qqq = pd.read_csv('./NASDAQ Composite Historical Data.csv')
tlt = pd.read_csv('./VUSTX.csv')

In [22]:
qqq['qqqR'] = qqq['R']
tlt['tltR'] = tlt['R']

In [23]:
qqq['Date'] = qqq['Date'].astype('datetime64[D]')
tlt['Date'] = tlt['Date'].astype('datetime64[D]')
qqq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5323 entries, 0 to 5322
Data columns (total 8 columns):
Date     5323 non-null datetime64[ns]
Price    5323 non-null object
Open     5323 non-null object
High     5323 non-null object
Low      5323 non-null object
Vol.     5323 non-null object
R        5323 non-null object
qqqR     5323 non-null object
dtypes: datetime64[ns](1), object(7)
memory usage: 332.8+ KB


In [24]:
dataSum = pd.merge(qqq, tlt, how='outer', left_on='Date', right_on='Date')[['Date','qqqR','tltR']]

In [25]:
type(dataSum['qqqR'].loc[0])

str

In [26]:
dataSum['qqqR'] = dataSum['qqqR'].str.replace('%','')
dataSum['tltR'] = dataSum['tltR'].str.replace('%','')
dataSum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 8514
Data columns (total 3 columns):
Date    8515 non-null datetime64[ns]
qqqR    5323 non-null object
tltR    8515 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 586.1+ KB


In [27]:
dataSum['qqqR']

0        0.01
1       -4.61
2        0.17
3       -2.77
4       -3.71
        ...  
8510      NaN
8511      NaN
8512      NaN
8513      NaN
8514      NaN
Name: qqqR, Length: 8515, dtype: object

In [28]:
dataSum[['qqqR','tltR']] = dataSum[['qqqR','tltR']].astype(float)

In [29]:
# dataSum['qqqR'] = dataSum['qqqR'].astype(float)
# dataSum['tltR'] = dataSum['tltR'].astype(float)

In [30]:
dataSum[['qqqR','tltR']] = dataSum[['qqqR','tltR']].fillna(0)

In [31]:
dataSum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 8514
Data columns (total 3 columns):
Date    8515 non-null datetime64[ns]
qqqR    8515 non-null float64
tltR    8515 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 586.1 KB


In [203]:
print(targetData.loc[targetData['tltR'] == 1.002] == 3)

    Date   qqqR   tltR
2  False  False  False


In [339]:
#if


# dataSum['qqqR'].head()
dataSumSorted = dataSum.sort_values('Date').set_index('Date').reset_index()
dataSumSorted.head(5)
tmpDataSumSorted = dataSumSorted
tmpDataSumSorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8515 entries, 0 to 8514
Data columns (total 3 columns):
Date    8515 non-null datetime64[ns]
qqqR    8515 non-null float64
tltR    8515 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 199.7 KB


In [340]:
# (dataSumSorted['Date'].loc[0] - pd.DateOffset(months=1)> > dataSumSorted['Date'].loc[0])
phaseArr = []
for idx, val in enumerate(tmpDataSumSorted['Date']):
    try:
        idxStartNum = 0;
        tmpIdx = idx + idxStartNum;
        targetDate = tmpDataSumSorted['Date'].loc[tmpIdx] 
        targetPhaseDate = tmpDataSumSorted['Date'].loc[tmpIdx] if (tmpIdx == idxStartNum) else tmpDataSumSorted['Date'].loc[tmpIdx-1]
        targetData = tmpDataSumSorted.loc[tmpDataSumSorted['Date'] <= targetDate].loc[tmpDataSumSorted['Date'] + pd.DateOffset(months=1) > targetDate]

        phaseOverCount = 0
        phaseOverCount = targetData['qqqR'].loc[targetData['qqqR'] < -3].count()

    # #     targetData
#         print(idx)
#         print(targetData)
    #     print(targetDate)
    #     print(targetPhaseDate)
    #     print(val)
    #     print(targetData.loc[targetData['Date'] == targetPhaseDate])
        isPhase3 = targetData['qqqR'].loc[targetData['Date'] == targetPhaseDate] == 3
        if(phaseOverCount >= 1 and isPhase3): # 수익률-3 이하이고, 페이즈 3일때
            phaseArr.append(3)
            tmpDataSumSorted.loc[tmpDataSumSorted['Date'] == targetDate, 'qqqR'] = 3
        else: # 수익률-3 이하이고, 페이즈 3아닐때
            phaseArr.append(1)
            tmpDataSumSorted.loc[tmpDataSumSorted['Date'] == targetDate, 'qqqR'] = 1
    except Exception as e:
        print(e)
        print(idx)
        print(val)
# print(len(phaseArr))
print(phaseArr)


The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3214
1999-02-04 00:00:00
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3215
1999-02-05 00:00:00
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3216
1999-02-08 00:00:00
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3217
1999-02-09 00:00:00
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3218
1999-02-10 00:00:00
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3219
1999-02-11 00:00:00
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3220
1999-02-12 00:00:00
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
3221
1999-02-16 00:00:00
The truth value of a Series is ambiguous

In [142]:
targetDate = tmpDataSumSorted['Date'].loc[10]
print(targetDate)
abc = tmpDataSumSorted['Date'].loc[tmpDataSumSorted['Date'] >= targetDate].loc[tmpDataSumSorted['Date']- pd.DateOffset(months=1) <= targetDate]
abc.head(50)

1986-06-03 00:00:00


10   1986-06-03
11   1986-06-04
12   1986-06-05
13   1986-06-06
14   1986-06-09
15   1986-06-10
16   1986-06-11
17   1986-06-12
18   1986-06-13
19   1986-06-16
20   1986-06-17
21   1986-06-18
22   1986-06-19
23   1986-06-20
24   1986-06-23
25   1986-06-24
26   1986-06-25
27   1986-06-26
28   1986-06-27
29   1986-06-30
30   1986-07-01
31   1986-07-02
Name: Date, dtype: datetime64[ns]

In [47]:
dataSum.describe()

,qqqR,tltR
count,8515.000000,8515.000000
mean,0.023692,0.032566
std,1.240583,0.658659
min,-9.670000,-5.689000
25%,-0.140000,-0.330000
50%,0.000000,0.000000
75%,0.320000,0.409000
max,14.170000,7.757000


In [18]:
# dataSum['R'] = phaseR
phaseR = []
for idx, val in enumerate(dataSum['phase']):
    for i in CONFIG['phaseList']:
        if i['phaseNum'] == val:            
            phaseR.append(i['phaseQqqPosition'] * dataSum['qqqR'].loc[idx] + i['phaseTltPosition'] * dataSum['tltR'].loc[idx])
# dataSum['phase'].value == CONFIG['phaseList'].value[phaseNum]
dataSum['resultR'] = phaseR
dataSum.sample(10)

KeyError: 'phase'

In [ ]:
print(dataSum['resultR'].head(20))

In [ ]:
CONFIG['phaseList']